In [211]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import  train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import Imputer   #处理缺失值的类

In [2]:
df_train=pd.read_csv('train (1).csv')

In [224]:
def preprocess(data):
    '''
       将标量数据转为为数值
    '''
    returnList=[]
    return_str_index=[]
    return_label=[]
    le_list=[]
    le_num=0
    for data_list in data: #遍历每一个数据行
        str_index=[]   #储存元素类型为字符串的元素下标
        temp_list=data_list.tolist()[0]  
        temp_label=[]  #储存元素类型为字符串的元素
        for i in range(0,len(temp_list)):
            if isinstance(temp_list[i],str):
                str_index.append(i)
        for index in str_index:
            temp_label.append(temp_list[index])
        le_list.append(preprocessing.LabelEncoder())
        le_list[le_num].fit(temp_label)
        for index in str_index:
            temp_list[index]=le_list[le_num].transform([temp_list[index]])[0] #将字符串替换为数值型数据 
        returnList.append(temp_list)  #将替换好的数据加入到列表中
        return_str_index.append(str_index)
        return_label.append(temp_label)
        le_num+=1
    return(np.matrix(returnList),return_str_index,return_label,le_list)
def preprocess_na(data):
    from sklearn.preprocessing import Imputer
    imp=Imputer(missing_values=np.nan,strategy='mean',axis=0)
    X=np.array([0,0])
    imp.fit(X)
    return(imp.transform(data))

In [255]:
df_X=np.matrix(df_train[list(df_train.columns)[2:len(df_train)]] )
df_Y=np.matrix(df_train[list(df_train.columns)[1]]).T
#将标量数据转为为数值
df_X,str_index_list,label_list,le_list=preprocess(df_X)
#处理缺失值,将缺失值填充为0
df_X[np.isnan(df_X)]=0  
#拆分训练集与验证集
train_X,valid_X,train_Y,valid_Y=train_test_split(df_X,df_Y,test_size=0.3,random_state=42) 
#构建决策树模型
clf=tree.DecisionTreeClassifier(criterion='entropy',max_features=None,min_samples_split=10,min_samples_leaf=5)
#训练模型
clf.fit(X=train_X,y=train_Y)
#使用训练好的模型进行预测
predict=clf.predict(X=valid_X)
#计算预测精度
accuracy=np.mean(np.matrix(predict)==valid_Y.reshape(-1))
print(accuracy)
print(clf.feature_importances_)
print

0.682835820896
[ 0.18337564  0.02106036  0.0132439   0.26184562  0.03730062  0.07457476
  0.02032874  0.33821623  0.          0.05005412]


In [256]:
help(tree.DecisionTreeClassifier)

Help on class DecisionTreeClassifier in module sklearn.tree.tree:

class DecisionTreeClassifier(BaseDecisionTree, sklearn.base.ClassifierMixin)
 |  A decision tree classifier.
 |  
 |  Read more in the :ref:`User Guide <tree>`.
 |  
 |  Parameters
 |  ----------
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |  
 |  splitter : string, optional (default="best")
 |      The strategy used to choose the split at each node. Supported
 |      strategies are "best" to choose the best split and "random" to choose
 |      the best random split.
 |  
 |  max_features : int, float, string or None, optional (default=None)
 |      The number of features to consider when looking for the best split:
 |  
 |          - If int, then consider `max_features` features at each split.
 |          - If float, then `max_features` is a percentage and
 |